In [15]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import time
import tracemalloc

from lib.cnnae_fully_convolutional import createLevel3FullyConvDropoutNet

%config InlineBackend.figure_formats = ['svg']
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
# load data
path_to_data = "./data/reference/"

X_test = np.load(f"{path_to_data}X.dat_test.npy")
Y_test = np.load(f"{path_to_data}Y.dat_test.npy")

In [18]:
# Add a channel dimension (for grayscale 1-channel data)
# X_test = np.expand_dims(X_test, axis=1)
# Y_test = np.expand_dims(Y_test, axis=1)

In [19]:
X_test_tensor = torch.tensor(X_test[:, np.newaxis], dtype=torch.float32, device=device)
Y_test_tensor = torch.tensor(Y_test[:, np.newaxis], dtype=torch.float32, device=device)

In [20]:
X_test_tensor.shape

torch.Size([31455, 1, 7, 7])

In [21]:
# create dataset
test_data = TensorDataset(X_test_tensor, Y_test_tensor)

batchsize = 64
test_loader = DataLoader(test_data, batch_size=batchsize, shuffle=True, drop_last=True)

In [22]:
len(test_loader.dataset)

31455

In [23]:
(len(test_loader.dataset) // batchsize) * batchsize

31424

In [24]:
X_test, Y_test = next(iter(test_loader))
X_test.shape

torch.Size([64, 1, 7, 7])

In [25]:
def measure_inference_time(model, test_loader):
    start_time = time.perf_counter()

    is_fully_connected = isinstance(model, LargeNNv2)

    with torch.no_grad():
        X_test, Y_test = next(iter(test_loader))
        if is_fully_connected:
            # Reshape (batch_size, 1, 7, 7) to (batch_size, 49) for LargeNNv2
            X_test = X_test.view(X_test.size(0), -1)
        model(X_test)

    # for X_test, Y_test in test_loader:
         

    total_time = time.perf_counter() - start_time
    avg_time = total_time / len(test_loader.dataset)
    return avg_time

In [26]:
def measure_memory_usage(model, test_loader):
    tracemalloc.start()

    is_fully_connected = isinstance(model, LargeNNv2)

    with torch.no_grad():
        X_test, Y_test = next(iter(test_loader))
        if is_fully_connected:
            # Reshape (batch_size, 1, 7, 7) to (batch_size, 49) for LargeNNv2
            X_test = X_test.view(X_test.size(0), -1)
        model(X_test)

    _, peak_memory = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    peak_memory_mb = peak_memory / (1024 * 1024)  # Convert to MB
    return peak_memory_mb

In [27]:
def evaluate_found_path(answer: torch.Tensor, Y_test: torch.Tensor):
    start = find_gate(answer, 0.6, True)
    end = find_gate(answer, 0.6, False)

    if start is None or end is None:
        return False

    current = (start[0], start[1])
    visited = set()
    visited.add(current)

    max_moves = (
        answer.shape[0] * answer.shape[1]
    )  # Consider the grid size for max moves
    # max_moves = 17

    for i in range(max_moves):
        brightest_neighbour = find_brightest_neighbour(answer, current, visited, Y_test)
        if brightest_neighbour is None:
            return False
        current = (brightest_neighbour[0], brightest_neighbour[1])
        visited.add(current)
        if current == end:
            return True
    return False


def find_gate(answer: torch.Tensor, epsillon: float, start: bool):
    rows, cols = answer.shape  # Get the actual number of rows and columns
    for row in range(rows):
        if start:
            if answer[row][0] > (3 - epsillon):
                return row, 0
        else:
            if answer[row][cols - 1] > (
                3 - epsillon
            ):  # Use the last column dynamically
                return row, cols - 1
    return None


def find_brightest_neighbour(
    answer: torch.Tensor,
    position: tuple[int, int],
    visited: set[tuple[int, int]],
    Y_test: torch.Tensor,
):
    rows, cols = answer.shape  # Get the actual number of rows and columns

    ind_up = (max(position[0] - 1, 0), position[1])
    ind_down = (min(position[0] + 1, rows - 1), position[1])
    ind_left = (position[0], max(position[1] - 1, 0))
    ind_right = (position[0], min(position[1] + 1, cols - 1))

    value_up = answer[ind_up[0]][ind_up[1]] if ind_up not in visited else -1
    value_down = answer[ind_down[0]][ind_down[1]] if ind_down not in visited else -1
    value_left = answer[ind_left[0]][ind_left[1]] if ind_left not in visited else -1
    value_right = answer[ind_right[0]][ind_right[1]] if ind_right not in visited else -1

    # Find the maximum value among the neighbours and return the corresponding index
    max_value = max(value_up, value_down, value_left, value_right)
    if max_value == -1:
        return None

    if max_value == value_up and Y_test[ind_up[0]][ind_up[1]] != 1.0:
        return ind_up
    elif max_value == value_down and Y_test[ind_down[0]][ind_down[1]] != 1.0:
        return ind_down
    elif max_value == value_left and Y_test[ind_left[0]][ind_left[1]] != 1.0:
        return ind_left
    elif max_value == value_right and Y_test[ind_right[0]][ind_right[1]] != 1.0:
        return ind_right

    return None


def evaluate_path_accuracy(model, test_loader: DataLoader):
    preds_ = torch.zeros(len(test_loader.dataset), dtype=bool)

    # Check the model type or input requirements
    is_fully_connected = isinstance(model, LargeNNv2)

    with torch.no_grad():
        X_test, Y_test = next(iter(test_loader))
        if is_fully_connected:
            # Reshape (batch_size, 1, 7, 7) to (batch_size, 49) for LargeNNv2
            X_test = X_test.view(X_test.size(0), -1)

        # Pass the input through the model
        Y_hat = model(X_test)

        for index, (y_hat, y_true) in enumerate(zip(Y_hat, Y_test)):
            # Reshape predictions and targets to 2D (7x7) for evaluation if needed
            if is_fully_connected:
                y_hat_reshaped = y_hat.view(7, 7)
            else:
                y_hat_reshaped = (
                    y_hat.squeeze()
                )  # For FCAE, remove channel dimension

            y_true_reshaped = y_true.view(7, 7)

            # Evaluate the path accuracy
            preds_[index] = evaluate_found_path(y_hat_reshaped, y_true_reshaped)

    # Calculate accuracy
    total = preds_.shape[0]
    correct = preds_.sum().item()
    correct_percentage = (correct / total) * 100

    return correct_percentage

In [28]:
def compare_models(model1, model2, test_loader):
    """
    Compare two models using metrics like path accuracy, path length error, inference time, and memory usage.
    """
    metrics = {}

    # Evaluate Model 1
    print("Evaluating Model 1:")
    model1.to(device)
    model1.eval()
    metrics["Model 1"] = {
        "Accuracy": evaluate_path_accuracy(model1, test_loader),
        "Inference Time (s)": measure_inference_time(model1, test_loader),
        "Memory Usage (MB)": measure_memory_usage(model1, test_loader),
        # "Path Length Errors": evaluate_path_length_errors(model1, test_loader),
    }

    # Evaluate Model 2
    print("\nEvaluating Model 2:")
    model2.to(device)
    model2.eval()
    metrics["Model 2"] = {
        "Accuracy": evaluate_path_accuracy(model2, test_loader),
        "Inference Time (s)": measure_inference_time(model2, test_loader),
        "Memory Usage (MB)": measure_memory_usage(model2, test_loader),
        # "Path Length Errors": evaluate_path_length_errors(model2, test_loader),
    }

    # Print Comparison
    print("\nComparison Summary:")
    for model_name, results in metrics.items():
        print(f"\n{model_name} Results:")
        print(f"  Accuracy: {results['Accuracy']:.2f}%")
        print(f"  Inference Time: {results['Inference Time (s)']:.6f} seconds")
        print(f"  Memory Usage: {results['Memory Usage (MB)']:.2f} MB")
        # print(
        #     f"  Avg Path Length Absolute Error: {results['Path Length Errors'][0]:.2f}"
        # )
        # print(
        #     f"  Avg Path Length Percentage Error: {results['Path Length Errors'][1]:.2f}%"
        # )

    return metrics

In [29]:
# Create FCNN
class LargeNNv2(nn.Module):
    def __init__(self):
        super(LargeNNv2, self).__init__()
        self.fc1 = nn.Linear(49, 1235)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(1235, 768)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(768, 532)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(532, 149)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(149, 98)
        self.relu5 = nn.ReLU()
        self.fc6 = nn.Linear(98, 49)  # Output layer

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.relu4(self.fc4(x))
        x = self.relu5(self.fc5(x))
        x = self.fc6(x)
        return x

fcnn = LargeNNv2()

fcnn.load_state_dict(torch.load("./reference/model/collaboratory/m_300.pth", weights_only=True))

<All keys matched successfully>

In [30]:
# Load models
fcae, _, _ = createLevel3FullyConvDropoutNet((15, 84, 167))

fcae.load_state_dict(
    torch.load(
        "./archive/Level3FullyConvDropoutNet_99.8647/net.pt",
        weights_only=True,
    )
)

<All keys matched successfully>

In [31]:
fcae.to("cuda")
X_test_tensor.to("cuda")
# output = fcae(X_test_tensor)
# correct_percentage = evaluate_path_accuracy(fcae, test_loader)

tensor([[[[0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [3., 0., 0.,  ..., 0., 0., 3.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 1., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          ...,
          [0., 1., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [3., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 3.],
          [0., 0., 0.,  ..., 1., 1., 0.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          ...,
          [3., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 0.,  ..., 0., 0., 1.]]],


        ...,


        [[[3., 1., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 1., 0.],
          [0., 1., 1.,  ..., 0., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 0.

In [32]:
dataset = TensorDataset(X_test_tensor, Y_test_tensor)
dataloader = DataLoader(dataset, batch_size=batchsize, shuffle=True, drop_last=True)

In [35]:
outputs = []
outputs_truth = []

for X_batch, Y_batch in dataloader:
    X_batch.to("cuda")
    # Y_batch.to("cuda")
    output = fcae(X_batch)
    outputs.append(output)
    outputs_truth.append(Y_batch)

result = torch.cat(outputs, dim=0)
result_truth = torch.cat(outputs_truth, dim=0)

In [39]:
result.shape

torch.Size([31424, 1, 7, 7])

In [37]:
output.shape

torch.Size([64, 1, 7, 7])

In [41]:
from lib.evaluate import evaluate_total_predictions_set

correct_percentage = evaluate_total_predictions_set(result.cpu(), result_truth.cpu(), verbose=True)


----------MODEL SUMMARY-----------
TOTAL SET SIZE:  31424
CORRECT GUESSES:  31031
TOTALING TO ACCURACY%:  98.74936354378818
------------------------------------




In [43]:
X_test_tensor_reshaped = X_test_tensor.view(X_test_tensor.size(0), 1, -1)
Y_test_tensor_reshaped = Y_test_tensor.view(Y_test_tensor.size(0), 1, -1)

In [44]:
X_test_tensor_reshaped.shape

torch.Size([31455, 1, 49])

In [49]:
dataset_reshaped = TensorDataset(X_test_tensor_reshaped, Y_test_tensor_reshaped)
dataloader_reshaped = DataLoader(dataset_reshaped, batch_size=batchsize, shuffle=True, drop_last=True)

In [50]:
outputs2 = []
outputs2_truth = []
fcnn.to("cuda")

for X_batch, Y_batch in dataloader_reshaped:
    X_batch.to("cuda")
    # Y_batch.to("cuda")
    output = fcnn(X_batch)
    outputs2.append(output)
    outputs2_truth.append(Y_batch)

result = torch.cat(outputs2, dim=0)
result_truth = torch.cat(outputs2_truth, dim=0)

In [69]:
# fcnn.to("cuda")
# X_test_tensor_reshaped.to("cuda")
# output2 = fcnn(X_test_tensor_reshaped)

In [54]:
result_truth.shape

torch.Size([31424, 1, 49])

In [55]:
y_hat_reshaped = result.view(result.size(0), 1, 7, 7)
y_hat_reshaped.shape

torch.Size([31424, 1, 7, 7])

In [56]:
y_truth_reshaped = result_truth.view(result_truth.size(0), 1, 7, 7)
y_truth_reshaped.shape

torch.Size([31424, 1, 7, 7])

In [57]:
correct_percentage2 = evaluate_total_predictions_set(
    y_hat_reshaped.cpu(), y_truth_reshaped.cpu(), verbose=True
)


----------MODEL SUMMARY-----------
TOTAL SET SIZE:  31424
CORRECT GUESSES:  31377
TOTALING TO ACCURACY%:  99.85043279022403
------------------------------------




In [16]:
# metrics = compare_models(fcae, fcnn, test_loader)

Evaluating Model 1:

Evaluating Model 2:

Comparison Summary:

Model 1 Results:
  Accuracy: 99.81%
  Inference Time: 0.000018 seconds
  Memory Usage: 10.60 MB

Model 2 Results:
  Accuracy: 99.85%
  Inference Time: 0.000012 seconds
  Memory Usage: 10.61 MB
